# QPM : Assignement 4

## Librairies

In [32]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import scipy.optimize

## Functions

In [33]:
'''
Function get_multi_timeseries:

Returns the full time series of selected stocks metrics (Open, High, Low, Close, Adj Close, Volume, etc.)
for a list of given stock tickers, over a specified time period and interval.

Inputs:
    - list_underlying: list of str ; tickers of the desired stocks.
    - startd / endd: str ; start and end dates (inclusive) defining the time range to retrieve, in format "YYYY-MM-DD".
    - metric: str or list-str ; name of the stock's metric(s) we want to select (close, open, high, ...).

Output:
    - DataFrame with:
        - index: pandas Timestamps (dates in ascending order),
        - columns: MultiIndex with first level = metrics (e.g., "Close"), 
                   second level = stock tickers.


Function: annual_mean_returns ; annual_volatility ; sharpe_ratio

Returns the annual mean, return and sharpe ratio of each stocks.

Input: - df: DataFrame (float) ; DataFrame of time series of close price of each stocks.

Output: DataFrame (float) ; 1 column, indexed by the stocks tickers.

Function: annual_cov

Return the covariance matrix of the stocks metrics.

Input: df: DataFrame ; time series of the stocks metrics.

Output: DataFrame ; covariance matrix.

'''

def get_multi_timeseries(list_underlying, start, end, metric) :
    return yf.download(list_underlying, start = start, end = end)[metric] 

def annual_mean_return(df) :
    return df.mean() * 12

def annual_volatility(df) :
    return df.std() * np.sqrt(12)

def sharpe_ratio(df) :
    return annual_mean_return(df) / annual_volatility(df)

def annual_cov(df) :
    return df.cov() * np.sqrt(12)

## Questions for Assignment 4

#### Q4.0 Prepare the data for this assignment.

In [34]:
URL = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
tickers = pd.read_html(URL)[0]['Symbol'].tolist()

start_date = "2000-01-01"
end_date = "2022-12-31"
interval = "1M"
tickers = ["MMM","AOS","ABT","ADM","ADBE","ADP","AES","AFL","A","AKAM"]

stocks = get_multi_timeseries(tickers, start = start_date, end = end_date, metric = "Close")
stocks = stocks.resample(interval).last() #resample the time series to get a monthly, annual, ... time serie. Specified with the 'interval' argument.
stocks

/var/folders/lr/7kzpljcn0zl52pt8zpscgfj40000gn/T/ipykernel_2659/773961763.py:38: FutureWarning: YF.download() has changed argument auto_adjust default to True
  return yf.download(list_underlying, start = start, end = end)[metric]
[*********************100%***********************]  10 of 10 completed


Ticker,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
Date,,,,,,,,,,
2000-01-31,39.712391,7.672882,13.668242,5.989039,21.976856,26.556358,6.542188,249.125000,2.096057,19.361580
2000-02-29,62.324856,7.775973,25.319607,5.150373,20.181650,27.778524,5.517327,261.250000,1.830732,18.355238
2000-03-31,62.399857,8.291423,27.638033,5.278331,22.398582,26.100628,6.875438,160.812500,1.910330,18.433302
2000-04-30,53.174873,9.102676,30.027847,5.086393,24.980799,29.808577,7.365866,98.875000,2.196066,18.030039
2000-05-31,44.174896,9.635524,27.948397,6.139838,25.532066,28.917831,7.813314,66.750000,2.242791,17.963594
...,...,...,...,...,...,...,...,...,...,...
2022-08-31,125.509529,97.241287,373.440002,80.814507,229.341492,22.753576,55.882000,90.279999,53.685345,92.748970
2022-09-30,118.952698,91.661629,275.200012,73.973473,213.161942,20.205530,52.853733,80.320000,46.200775,82.418663
2022-10-31,135.628052,94.168991,318.500000,89.172737,227.778595,23.528843,61.233212,88.330002,52.400021,93.823021


# Q1 Compute the weigths of the tangency portfolio and of the Minimum variance portfolio.

In [35]:
#Compute the log returns
log_returns = np.log(stocks).diff(1).iloc[1:,:]
log_returns

Ticker,A,ABT,ADBE,ADM,ADP,AES,AFL,AKAM,AOS,MMM
Date,,,,,,,,,,
2000-02-29,0.450697,0.013346,0.616504,-0.150862,-0.085216,0.044994,-0.170378,0.047523,-0.135342,-0.053376
2000-03-31,0.001203,0.064183,0.087614,0.024541,0.104224,-0.062304,0.220062,-0.485239,0.042560,0.004244
2000-04-30,-0.159977,0.093347,0.082932,-0.037041,0.109110,0.132837,0.068901,-0.486383,0.139392,-0.022120
2000-05-31,-0.185429,0.056888,-0.071765,0.188229,0.021828,-0.030338,0.058973,-0.392902,0.021054,-0.003692
2000-06-30,0.001696,0.090970,0.144227,-0.196028,-0.024957,0.044826,-0.117934,0.575935,-0.003728,-0.032596
...,...,...,...,...,...,...,...,...,...,...
2022-08-31,-0.044604,-0.058554,-0.093692,0.064675,0.013552,0.135723,0.042781,-0.063721,-0.114056,-0.131211
2022-09-30,-0.053656,-0.059092,-0.305259,-0.088450,-0.073160,-0.118766,-0.055714,-0.116897,-0.150143,-0.118085
2022-10-31,0.131190,0.026987,0.146124,0.186869,0.066322,0.152271,0.147161,0.095061,0.125910,0.129598


The weights of the Mean-Variance Portfolio (MVP) — also called the tangency portfolio — are given by:

$$
w_{\text{MVP}} = \frac{\Sigma^{-1} \mu}{\mathbf{1}^\top \Sigma^{-1} \mu}
$$

where:
- $\Sigma$ is the covariance matrix of asset returns,  
- $\mu$ is the vector of expected returns of the assets,  
- $\mathbf{1}$ is a column vector of ones (used to normalize the weights so they sum to 1).

In [36]:
#MVP
log_returns_est = log_returns.iloc[:60,:]
mu_est = annual_mean_return(log_returns_est)
cov_est = annual_cov(log_returns_est)

def MVP(ret) : #Tangency portfolio
    mu = annual_mean_return(ret)
    n = len(mu)
    cov = annual_cov(ret)
    #gamma_inv = (muT-rf) / (np.dot((mu - np.zeros(n)).T ,np.dot(np.linalg.inv(cov), mu - np.zeros(n))))
    weights = np.dot(np.linalg.inv(cov),mu)  / np.dot(np.ones(n).T, np.dot(np.linalg.inv(cov), mu))
    return weights

MVP_10 = MVP(log_returns_est)

#Sanity check :
print(np.dot(MVP_10.T, np.ones(len(MVP_10))))
print(np.dot(MVP_10.T, mu_est))

0.9999999999999998
0.32387927584801546


The weights of the Global Minimum Variance (GMV) portfolio are given by:

$$
w_{\text{GMV}} = \frac{\Sigma^{-1} \mathbf{1}}{\mathbf{1}^\top \Sigma^{-1} \mathbf{1}}
$$

where:
- $\Sigma$ is the covariance matrix of asset returns,  
- $\mathbf{1}$ is a column vector of ones (of length equal to the number of assets).

In [37]:
#Global Minimum Variance Portfolio
def GMV(ret) :
    cov = ret.cov()
    num = np.dot(np.linalg.inv(cov), np.ones(len(cov)))
    den = np.dot(np.ones(len(cov)), np.dot(np.linalg.inv(cov), np.ones(len(cov))))
    return num /den
Gmv = GMV(log_returns_est)

#Sanity check
print(np.dot(Gmv.T, np.dot(log_returns_est.cov(), Gmv)) - np.dot(MVP_10.T, np.dot(cov_est, MVP_10)) < 0) #check if the GMV variance is lower than the MVP variance

True


# Q2. Compute the weights of the Global Minimum Variance and Mean-Variance portfolios using a rolling estimation window of 60 months.

In [38]:
Rolling_GMV = pd.DataFrame(index = log_returns.index, columns= log_returns.columns)
Rolling_MVP = pd.DataFrame(index = log_returns.index, columns= log_returns.columns)

T_est = 60
for i in log_returns.index[60 :] :
    start_date = i - pd.DateOffset(months = 60)
    end_date = i - pd.DateOffset(months = 1)
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    Est_window = log_returns.loc[start_date : end_date, :]
    Rolling_GMV.loc[i,:] = GMV(Est_window)
    Rolling_MVP.loc[i,:] = MVP(Est_window)
Rolling_GMV = Rolling_GMV.dropna()
Rolling_MVP = Rolling_MVP.dropna()

# Q3. Using rolling estimated portfolio weights (GMV and MVP), compute the ex-post returns and annualized Sharpe ratios of each portfolio.

In [39]:
GMV_returns = (Rolling_GMV * log_returns.iloc[60:,:]).sum(axis = 1)
MVP_returns = (Rolling_MVP * log_returns.iloc[60:,:]).sum(axis = 1)

In [40]:
GMV_sharpe = np.mean(GMV_returns) / np.std(GMV_returns) * np.sqrt(12)
MVP_sharpe = np.mean(MVP_returns) / np.std(MVP_returns) * np.sqrt(12)

print("MVP :", MVP_sharpe)
print("GMV :", GMV_sharpe)

MVP : -0.1077038890983879
GMV : 0.6862791254676474
